In [ ]:
# Importieren der erforderlichen Module
from tkinter import *
from tkinter import messagebox
import random

#Herstellung des Spielfenster/Interface
window = Tk()
window.geometry("1200x1200")
window.title("4 Gewinnt")

# Spieler und Zufallsauswahl für den Startspieler
players = ["ai","hu"]
player = random.choice(players)
aiMark = players[0]
humanMark = players[1]

# Funktion zum Beenden des Spiels     
def exit_game():
    window.destroy() 
    
# Definition einer Funktion, um den Zug der KI am Anfang zu behandeln
def place_x():
    aiTurn()

   
    
# Definition einer Funktion, um ein neues Spiel zu starten
def new_game():
    global player, count, board, board_v2

    player = player = random.choice(players)
    label.config(text=player+"'s turn")
    
    for i in range (7):
        for j in range (6):
            buttons[j][i].config(text="", bg="#F0F0F0")
    
    board = [
    ["","","","","","",""],
    ["","","","","","",""],
    ["","","","","","",""],
    ["","","","","","",""],
    ["","","","","","",""],
    ["","","","","","",""]]
    
    board_v2 = [
    ["","","","","","",""],
    ["","","","","","",""],
    ["","","","","","",""],
    ["","","","","","",""],
    ["","","","","","",""],
    [1,1,1,1,1,1,1]]
    
    count=0
    
    enable_buttons()
    

    
#Herstellung eines Frames und einfügen in dem Hauptfenster
frame = Frame(window)
frame.pack()

# Text für die Benutzeroberfläche
label = Label(text=player + "'s turn", font=('consolas',40))
label.pack(side="top")

# Schaltfläche zum Neustarten des Spiels
reset_button = Button(text="restart", font=('consolas',20), bg="plum1", command=new_game)
reset_button.pack(side="right")

# Schaltfläche zum Beenden des Spiels
exit_button = Button(text="exit", font=('consolas',20), bg="plum1", command=exit_game)
exit_button.pack(side="right")

# Schaltfläche zum Starten des Zuges der KI
startforai_button = Button(text="start for the ai", font=('consolas',20), bg='plum1', command=place_x)
startforai_button.pack(side="left")

# Darstellen des Spielfeldes mit den 42 Buttons
buttons = [[0,0,0,0,0,0,0],
           [0,0,0,0,0,0,0],
           [0,0,0,0,0,0,0],
           [0,0,0,0,0,0,0],
           [0,0,0,0,0,0,0],
           [0,0,0,0,0,0,0]]

board = [["","","","","","",""],
        ["","","","","","",""],
        ["","","","","","",""],
        ["","","","","","",""],
        ["","","","","","",""],
        ["","","","","","",""]]
board_v2 = [
    ["","","","","","",""],
    ["","","","","","",""],
    ["","","","","","",""],
    ["","","","","","",""],
    ["","","","","","",""],
    [1,1,1,1,1,1,1]]

#Herstellung eines 6x7-Raster von Buttons im frame
for row in range(6):
    for column in range(7):
        buttons[row][column] = Button(frame, text="",font=('consolas',40),bg="#F0F0F0", width=3, height=1,relief="groove",
                                      command= lambda row=row, column=column: next_turn(row,column))
        buttons[row][column].grid(row=row,column=column)

        
for i in range (7):        
    buttons[0][i]['state'] = DISABLED
    buttons[1][i]['state'] = DISABLED
    buttons[2][i]['state'] = DISABLED
    buttons[3][i]['state'] = DISABLED
    buttons[4][i]['state'] = DISABLED


count=0

Tie="Tie"
 

# depth:
depth = 6
# AlphaBeta-Algorithmus für die KI
def alphabeta(node, depth, alpha, beta, maximizingPlayer):
    global counter, buttons
    emptyCells = [(row, column) for row in range(6) for column in range(7) if board[row][column] == "" and board_v2[row][column] == 1]
    moves = [(row,column) for row in range(6) for column in range(7) if buttons[row][column].cget("bg") == "#F0F0F0" and buttons[row][column]['state'] == NORMAL]
    move = random.choice(moves)
    if depth == 0 or is_terminal(node) == True:
        return evaluate(node, depth), move
    
    if maximizingPlayer:
        max_scores = [(float('-inf'), None)] * 3

        for row,column in emptyCells:
            board[row][column] = aiMark
            if row > 0:
                board_v2[row-1][column] = 1
            score, _ = alphabeta(board, depth - 1 ,alpha, beta, False)
            board[row][column] = ""
            if row > 0:
                board_v2[row-1][column] = ""
            if score > max_scores[0][0]:
                max_scores[0] = (score, (row, column))
                max_scores.sort()

                alpha = max(alpha, max_scores[0][0])
                if alpha >= beta:
                    break

        if all(score == max_scores[0][0] for score, _ in max_scores):
            _, best_move = random.choice(max_scores)
        else:
            _, best_move = max(max_scores)

        return max_scores[-1][0], best_move
        
    else:
        min_score = float('inf')
        best_move = None
        for row,column in emptyCells:
            board[row][column] = humanMark
            if row > 0:
                board_v2[row-1][column] = 1
            score, _ = alphabeta(board, depth - 1 ,alpha, beta, True)
            board[row][column] = ""
            if row > 0:
                board_v2[row-1][column] = ""
            if score < min_score:
                min_score = score
                best_move = (row, column)
                beta = min(beta, min_score)
                if beta <= alpha:
                    break
        return min_score, best_move

# Überprüfen, ob das Spiel in einem Endzustand ist
def is_terminal(node):
    if check_winner(node) == 'ai': 
        return True
    if check_winner(node) == 'hu':
        return True
    if check_winner(node) == "Tie in node": 
        return True
    else:
        return False
    
    
# Bewertung des aktuellen Spielzustands und Zuweisung einer Punktzahl
def evaluate(node, depth): 
    
    if check_winner(node) == 'ai':
            return 1000 + depth
    elif check_winner(node) == 'hu':
            return -1000 - depth
    elif check_winner(node) == "Tie in node":
            return 0
    else:
        eval = 0
        for row in range(6):
            for column in range (7):
                if node[row][column] == aiMark:
                        if (row == 0 and column == 0) or (row == 0 and column == 6) or(row == 5 and column == 0) or(row == 5 and column == 6):
                            eval += 3
                        if (row == 0 and column == 1) or (row == 0 and column == 5) or (row == 1 and column == 0) or (row == 1 and column == 6) or (row == 4 and column == 0) or (row == 4 and column == 6) or (row == 5 and column == 1) or (row == 5 and column == 5):
                            eval += 4 
                        if (row == 0 and column == 2) or (row == 0 and column == 4) or (row == 2 and column == 0) or (row == 2 and column == 6) or (row == 3 and column == 0) or (row == 3 and column == 6) or (row == 5 and column == 2) or (row == 5 and column == 4):
                            eval += 5
                        if (row == 1 and column == 1) or (row == 1 and column == 5) or(row == 4 and column == 1) or(row == 4 and column == 5):
                            eval += 6
                        if (row == 0 and column == 3) or (row == 5 and column == 3):
                            eval += 7
                        if (row == 1 and column == 2) or (row == 1 and column == 4) or (row == 2 and column == 1) or (row == 2 and column == 5) or (row == 3 and column == 1) or (row == 3 and column == 5) or (row == 4 and column == 2) or (row == 4 and column == 4):
                            eval += 8
                        if (row == 1 and column == 3) or (row == 4 and column == 3):
                            eval += 10
                        if (row == 2 and column == 2) or (row == 2 and column == 4) or(row == 3 and column == 2) or(row == 3 and column == 4):
                            eval += 11
                        if (row == 2 and column == 3) or (row == 3 and column == 3):
                            eval += 13
                        

                elif node[row][column] == humanMark:
                        if (row == 0 and column == 0) or (row == 0 and column == 6) or(row == 5 and column == 0) or(row == 5 and column == 6):
                            eval -= 3
                        if (row == 0 and column == 1) or (row == 0 and column == 5) or (row == 1 and column == 0) or (row == 1 and column == 6) or (row == 4 and column == 0) or (row == 4 and column == 6) or (row == 5 and column == 1) or (row == 5 and column == 5):
                            eval -= 4 
                        if (row == 0 and column == 2) or (row == 0 and column == 4) or (row == 2 and column == 0) or (row == 2 and column == 6) or (row == 3 and column == 0) or (row == 3 and column == 6) or (row == 5 and column == 2) or (row == 5 and column == 4):
                            eval -= 5
                        if (row == 1 and column == 1) or (row == 1 and column == 5) or(row == 4 and column == 1) or(row == 4 and column == 5):
                            eval -= 6
                        if (row == 0 and column == 3) or (row == 5 and column == 3):
                            eval -= 7
                        if (row == 1 and column == 2) or (row == 1 and column == 4) or (row == 2 and column == 1) or (row == 2 and column == 5) or (row == 3 and column == 1) or (row == 3 and column == 5) or (row == 4 and column == 2) or (row == 4 and column == 4):
                            eval -= 8
                        if (row == 1 and column == 3) or (row == 4 and column == 3):
                            eval -= 10
                        if (row == 2 and column == 2) or (row == 2 and column == 4) or(row == 3 and column == 2) or(row == 3 and column == 4):
                            eval -= 11
                        if (row == 2 and column == 3) or (row == 3 and column == 3):
                            eval -= 13

        return eval

# Behandeln des Zugs des KI-Spielers
def aiTurn():
    emptyCells = [(row, column) for row in range(6) for column in range(7) if board[row][column] == "" and board_v2[row][column] == 1]
    global count, player
    if emptyCells:
        score, best_move = alphabeta(board, depth,float('-inf'), float('inf'), True)
        row, column = best_move
        board[row][column] = aiMark  
        buttons[row][column].config(bg="red")
        buttons[row-1][column]["state"] = NORMAL
        board_v2[row-1][column] = 1
    
    count+= 1
    aiPLayed = True
    if check_winner(board) is False:
        player = players[1] 
        label.config(text=player + "'s turn")   
        aiPlayed = True
    elif check_winner(board) == 'ai':
        label.config(text="Oh no " + player + " wins")
        disable_buttons()
    elif check_winner(board) == "Tie":
        label.config(text="It's a Tie")
        disable_buttons()      

# Behandeln des Zugs des menschlichen Spielers        
def next_turn(row, column):
    global player, winner, count , aiTurn
    if player == players[1]:
        if buttons[row][column].cget("bg") == "#F0F0F0":
                board[row][column] = player
                buttons[row][column].config(bg="yellow")
                buttons[row-1][column]['state'] = NORMAL
                board_v2[row-1][column] = 1
                count += 1
                if check_winner(board) is False:
                    player = players[0]
                    label.config(text=player + "'s turn")
                    window.after(200, aiTurn)
                elif check_winner(board) == "hu":
                    label.config(text= "Hurray "  +  player + " wins")
                    disable_buttons()
                elif check_winner(board) == "Tie":
                    label.config(text="It's a Tie")
                    disable_buttons()
        else:
                messagebox.showerror(title=None, message='This square is already taken. Please choose another one')
    else: 
        return None
 




    
#Defintion um die Knöpfe auszuschalten 
def disable_buttons():
    for i in range (7):
        buttons[0][i]['state'] = DISABLED
        buttons[1][i]['state'] = DISABLED
        buttons[2][i]['state'] = DISABLED
        buttons[3][i]['state'] = DISABLED
        buttons[4][i]['state'] = DISABLED
        buttons[5][i]['state'] = DISABLED
        

#Defintion um die Knöpfe anzuschalten    
def enable_buttons():
    for i in range (7):
        buttons[0][i]['state'] = DISABLED
        buttons[1][i]['state'] = DISABLED
        buttons[2][i]['state'] = DISABLED
        buttons[3][i]['state'] = DISABLED
        buttons[4][i]['state'] = DISABLED
        buttons[5][i]['state'] = DISABLED
        buttons[5][i]['state'] = NORMAL

        
# Funktion zum Überprüfen des Gewinners  
def check_winner(board):
    global count
    
    emptyCells = [(row, column) for row in range(6) for column in range(7) if board[row][column] == "" and board_v2[row][column] == 1]
    
    if board[0][0] == board[0][1] == board[0][2] == board[0][3] != '':
        if board[0][0] == aiMark:
            return 'ai'
        if board[0][0] == humanMark: 
            return 'hu'

    elif board[0][4] == board[0][1] == board[0][2] == board[0][3] != '':
        if board[0][4] == aiMark:
            return 'ai'
        if board[0][4] == humanMark:
            return 'hu'
     
    elif board[0][4] == board[0][5] == board[0][2] == board[0][3] != '':
        if board[0][4] == aiMark:
            return 'ai'
        if board[0][4] == humanMark:
            return 'hu'
    
    elif board[0][4] == board[0][5] == board[0][6] == board[0][3] != '':
        if board[0][4] == aiMark:
            return 'ai'
        if board[0][4] == humanMark:
            return 'hu'
    
    elif board[1][0] == board[1][1] == board[1][2] == board[1][3] != '':
        if board[1][0] == aiMark:
            return 'ai'
        if board[1][0] == humanMark:
            return 'hu'

    elif board[1][4] == board[1][1] == board[1][2] == board[1][3] != '':
        if board[1][4] == aiMark:
            return 'ai'
        if board[1][4] == humanMark:
            return 'hu'

    elif board[1][4] == board[1][5] == board[1][2] == board[1][3] != '':
        if board[1][4] == aiMark:
            return 'ai'
        if board[1][4] == humanMark:
            return 'hu'

    elif board[1][4] == board[1][5] == board[1][6] == board[1][3] != '':
        if board[1][4] == aiMark:
            return 'ai'
        if board[1][4] == humanMark:
            return 'hu'

    elif board[2][0] == board[2][1] == board[2][2] == board[2][3] != '':
        if board[2][0] == aiMark:
            return 'ai'
        if board[2][0] == humanMark:
            return 'hu'

    elif board[2][4] == board[2][1] == board[2][2] == board[2][3] != '':
        if board[2][4] == aiMark:
            return 'ai'
        if board[2][4] == humanMark:
            return 'hu'

    elif board[2][4] == board[2][5] == board[2][2] == board[2][3] != '':
        if board[2][4] == aiMark:
            return 'ai'
        if board[2][4] == humanMark:
            return 'hu'

    elif board[2][4] == board[2][5] == board[2][6] == board[2][3] != '':
        if board[2][4] == aiMark:
            return 'ai'
        if board[2][4] == humanMark:
            return 'hu'

    elif board[3][0] == board[3][1] == board[3][2] == board[3][3] != '':
        if board[3][0] == aiMark:
            return 'ai'
        if board[3][0] == humanMark:
            return 'hu'

    elif board[3][4] == board[3][1] == board[3][2] == board[3][3] != '':
        if board[3][4] == aiMark:
            return 'ai'
        if board[3][4] == humanMark:
            return 'hu'

    elif board[3][4] == board[3][5] == board[3][2] == board[3][3] != '':
        if board[3][4] == aiMark:
            return 'ai'
        if board[3][4] == humanMark:
            return 'hu'

    elif board[3][4] == board[3][5] == board[3][6] == board[3][3] != '':
        if board[3][4] == aiMark:
            return 'ai'
        if board[3][4] == humanMark:
            return 'hu'

    elif board[4][0] == board[4][1] == board[4][2] == board[4][3] != '':
        if board[4][0] == aiMark:
            return 'ai'
        if board[4][0] == humanMark:
            return 'hu'

    elif board[4][4] == board[4][1] == board[4][2] == board[4][3] != '':
        if board[4][4] == aiMark:
            return 'ai'
        if board[4][4] == humanMark:
            return 'hu'
     
    elif board[4][4] == board[4][5] == board[4][2] == board[4][3] != '':
        if board[4][4] == aiMark:
            return 'ai'
        if board[4][4] == humanMark:
            return 'hu'
    
    elif board[4][4] == board[4][5] == board[4][6] == board[4][3] != '':
        if board[4][4] == aiMark:
            return 'ai'
        if board[4][4] == humanMark:
            return 'hu'
    
    elif board[5][0] == board[5][1] == board[5][2] == board[5][3] != '':
        if board[5][0] == aiMark:
            return 'ai'
        if board[5][0] == humanMark:
            return 'hu'
    
    elif board[5][4] == board[5][1] == board[5][2] == board[5][3] != '':
        if board[5][4] == aiMark:
            return 'ai'
        if board[5][4] == humanMark:
            return 'hu'
     
    elif board[5][4] == board[5][5] == board[5][2] == board[5][3] != '':
        if board[5][4] == aiMark:
            return 'ai'
        if board[5][4] == humanMark:
            return 'hu'
    
    elif board[5][4] == board[5][5] == board[5][6] == board[5][3] != '':
        if board[5][4] == aiMark:
            return 'ai'
        if board[5][4] == humanMark:
            return 'hu'
    #Vertikal
    elif board[0][0] == board[1][0] == board[2][0] == board[3][0] != '':
        if board[0][0] == aiMark:
            return 'ai'
        if board[0][0] == humanMark:
            return 'hu'

    elif board[4][0] == board[1][0] == board[2][0] == board[3][0] != '':
        if board[4][0] == aiMark:
            return 'ai'
        if board[4][0] == humanMark:
            return 'hu' 

    elif board[4][0] == board[5][0] == board[2][0] == board[3][0] != '':
        if board[4][0] == aiMark:
            return 'ai'
        if board[4][0] == humanMark:
            return 'hu'   
    
    elif board[0][1] == board[1][1] == board[2][1] == board[3][1] != '':
        if board[0][1] == aiMark:
            return 'ai'
        if board[0][1] == humanMark:
            return 'hu'

    elif board[4][1] == board[1][1] == board[2][1] == board[3][1] != '':
        if board[4][1] == aiMark:
            return 'ai'
        if board[4][1] == humanMark:
            return 'hu'

    elif board[4][1] == board[5][1] == board[2][1] == board[3][1] != '':
        if board[4][1] == aiMark:
            return 'ai'
        if board[4][1] == humanMark:
            return 'hu'  

    elif board[0][2] == board[1][2] == board[2][2] == board[3][2] != '':
        if board[0][2] == aiMark:
            return 'ai'
        if board[0][2] == humanMark:
            return 'hu'

    elif board[4][2] == board[1][2] == board[2][2] == board[3][2] != '':
        if board[4][2] == aiMark:
            return 'ai'
        if board[4][2] == humanMark:
            return 'hu' 

    elif board[4][2] == board[5][2] == board[2][2] == board[3][2] != '':
        if board[4][2] == aiMark:
            return 'ai'
        if board[4][2] == humanMark:
            return 'hu'

    elif board[0][3] == board[1][3] == board[2][3] == board[3][3] != '':
        if board[0][3] == aiMark:
            return 'ai'
        if board[0][3] == humanMark:
            return 'hu'

    elif board[4][3] == board[1][3] == board[2][3] == board[3][3] != '':
        if board[4][3] == aiMark:
            return 'ai'
        if board[4][3] == humanMark:
            return 'hu'

    elif board[4][3] == board[5][3] == board[2][3] == board[3][3] != '':
        if board[4][3] == aiMark:
            return 'ai'
        if board[4][3] == humanMark:
            return 'hu'  

    elif board[0][4] == board[1][4] == board[2][4] == board[3][4] != '':
        if board[0][4] == aiMark:
            return 'ai'
        if board[0][4] == humanMark:
            return 'hu'

    elif board[4][4] == board[1][4] == board[2][4] == board[3][4] != '':
        if board[4][4] == aiMark:
            return 'ai'
        if board[4][4] == humanMark:
            return 'hu'

    elif board[4][4] == board[5][4] == board[2][4] == board[3][4] != '':
        if board[4][4] == aiMark:
            return 'ai'
        if board[4][4] == humanMark:
            return 'hu'   

    elif board[0][5] == board[1][5] == board[2][5] == board[3][5] != '':
        if board[0][5] == aiMark:
            return 'ai'
        if board[0][5] == humanMark:
            return 'hu'

    elif board[4][5] == board[1][5] == board[2][5] == board[3][5] != '':
        if board[4][5] == aiMark:
            return 'ai'
        if board[4][5] == humanMark:
            return 'hu' 

    elif board[4][5] == board[5][5] == board[2][5] == board[3][5] != '':
        if board[4][5] == aiMark:
            return 'ai'
        if board[4][5] == humanMark:
            return 'hu'    

    elif board[0][6] == board[1][6] == board[2][6] == board[3][6] != '':
        if board[0][6] == aiMark:
            return 'ai'
        if board[0][6] == humanMark:
            return 'hu'

    elif board[4][6] == board[1][6] == board[2][6] == board[3][6] != '':
        if board[4][6] == aiMark:
            return 'ai'
        if board[4][6] == humanMark:
            return 'hu'

    elif board[4][6] == board[5][6] == board[2][6] == board[3][6] != '':
        if board[4][6] == aiMark:
            return 'ai'
        if board[4][6] == humanMark:
            return 'hu'

#Diagonal von links nach rechts
    
    elif board[5][0] == board[4][1] == board[3][2] == board[2][3] != '':
        if board[5][0] == aiMark:
            return 'ai'
        if board[5][0] == humanMark:
            return 'hu'

    elif board[5][1] == board[4][2] == board[3][3] == board[2][4] != '':
        if board[5][1] == aiMark:
            return 'ai'
        if board[5][1] == humanMark:
            return 'hu'

    elif board[5][2] == board[4][3] == board[3][4] == board[2][5] != '':
        if board[5][2] == aiMark:
            return 'ai'
        if board[5][2] == humanMark:
            return 'hu'

    elif board[5][3] == board[4][4] == board[3][5] == board[2][6] != '':
        if board[5][3] == aiMark:
            return 'ai'
        if board[5][3] == humanMark:
            return 'hu'

    elif board[4][0] == board[3][1] == board[2][2] == board[1][3] != '':
        if board[4][0] == aiMark:
            return 'ai'
        if board[4][0] == humanMark:
            return 'hu'

    elif board[4][1] == board[3][2] == board[2][3] == board[1][4] != '':
        if board[4][1] == aiMark:
            return 'ai'
        if board[4][1] == humanMark:
            return 'hu'

    elif board[4][2] == board[3][3] == board[2][4] == board[1][5] != '':
        if board[4][2] == aiMark:
            return 'ai'
        if board[4][2] == humanMark:
            return 'hu'

    elif board[4][3] == board[3][4] == board[2][5] == board[1][6] != '':
        if board[4][3] == aiMark:
            return 'ai'
        if board[4][3] == humanMark:
            return 'hu'

    elif board[3][0] == board[2][1] == board[1][2] == board[0][3] != '':
        if board[3][0] == aiMark:
            return 'ai'
        if board[3][0] == humanMark:
            return 'hu'

    elif board[3][1] == board[2][2] == board[1][3] == board[0][4] != '':
        if board[3][1] == aiMark:
            return 'ai'
        if board[3][1] == humanMark:
            return 'hu'

    elif board[3][2] == board[2][3] == board[1][4] == board[0][5] != '':
        if board[3][2] == aiMark:
            return 'ai'
        if board[3][2] == humanMark:
            return 'hu'

    elif board[3][3] == board[2][4] == board[1][5] == board[0][6] != '':
        if board[3][3] == aiMark:
            return 'ai'
        if board[3][3] == humanMark:
            return 'hu'

    #Diagonal von rechts nach links
    
    elif board[5][6] == board[4][5] == board[3][4] == board[2][3] != '':
        if board[5][6] == aiMark:
            return 'ai'
        if board[5][6] == humanMark:
            return 'hu' 

    elif board[5][5] == board[4][4] == board[3][3] == board[2][2] != '':
        if board[5][5] == aiMark:
            return 'ai'
        if board[5][5] == humanMark:
            return 'hu' 

    elif board[5][4] == board[4][3] == board[3][2] == board[2][1] != '':
        if board[5][4] == aiMark:
            return 'ai'
        if board[5][4] == humanMark:
            return 'hu'

    elif board[5][3] == board[4][2] == board[3][1] == board[2][0] != '':
        if board[5][3] == aiMark:
            return 'ai'
        if board[5][3] == humanMark:
            return 'hu'

    elif board[4][6] == board[3][5] == board[2][4] == board[1][3] != '':
        if board[4][6] == aiMark:
            return 'ai'
        if board[4][6] == humanMark:
            return 'hu'

    elif board[4][5] == board[3][4] == board[2][3] == board[1][2] != '':
        if board[4][5] == aiMark:
            return 'ai'
        if board[4][5] == humanMark:
            return 'hu'

    elif board[4][4] == board[3][3] == board[2][2] == board[1][1] != '':
        if board[4][4] == aiMark:
            return 'ai'
        if board[4][4] == humanMark:
            return 'hu'

    elif board[4][3] == board[3][2] == board[2][1] == board[1][0] != '':
        if board[4][3] == aiMark:
            return 'ai'
        if board[4][3] == humanMark:
            return 'hu'

    elif board[3][6] == board[2][5] == board[1][4] == board[0][3] != '':
        if board[3][6] == aiMark:
            return 'ai'
        if board[3][6] == humanMark:
            return 'hu'

    elif board[3][5] == board[2][4] == board[1][3] == board[0][2] != '':
        if board[3][5] == aiMark:
            return 'ai'
        if board[3][5] == humanMark:
            return 'hu'

    elif board[3][4] == board[2][3] == board[1][2] == board[0][1] != '':
        if board[3][4] == aiMark:
            return 'ai'
        if board[3][4] == humanMark:
            return 'hu' 

    elif board[3][3] == board[2][2] == board[1][1] == board[0][0] != '':
        if board[3][3] == aiMark:
            return 'ai'
        if board[3][3] == humanMark:
            return 'hu'
                                                               
    if count == 42:
            return "Tie"
    elif not emptyCells:
            return "Tie in node"
    else: 
            return False
    



# Hauptschleife für das Tkinter-Fenster
window.mainloop()